In [6]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
from bs4 import BeautifulSoup
import src.job_scrape_funcs as scrape
import src.analysis as ana
import src.models as models
from datetime import datetime
from pathlib import Path

def job_pipeline(job_title, location, post_time, pages, save_path, resume_path, word_scores):
    #TODO: try except statements
    # Scrape jobs:
    job_data = scrape.scrape_jobs(pages=pages, job_title=job_title, location=location, post_time=post_time)
    date = datetime.today().strftime("%Y-%m-%d")

    # Save:
    index = 1
    full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"
    while (Path.cwd()/full_path).exists():
        index += 1
        full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"

    print("Saving to ", full_path)
    with open(full_path, 'w', encoding='utf-8') as f:
        json.dump(job_data, f, ensure_ascii=False, indent=4)

    # Find matches:
    with open(full_path, 'r', encoding="utf-8") as f:
        data = json.load(f)

    raw_df = pd.DataFrame(data["jobs"])
    df = ana.find_job_match(models.embed_model, models.nlp_model, raw_df, resume_path, models.stop_words, word_scores)
    print("Generating report:\n\n")
    html = ana.generate_html_report(df)
    soup = BeautifulSoup(html)
    with open("output1.html", "w") as file:
        file.write(str(soup))
    return df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
word_scores = {
    "phd": 1.3,
    "python": 1.1,
    "senior": 0.8,
}

In [8]:
df = job_pipeline(job_title="Data Scientist", 
                  location="Chicago", 
                  post_time=3, 
                  pages=40,
                  save_path="data/job_queries/", 
                  resume_path="data/resume.pdf",
                  word_scores=word_scores)

Scraping page 0
10 scraped successfully.
Scraping page 1
Job filtered out
19 scraped successfully.
Scraping page 2
Job filtered out
Job filtered out
27 scraped successfully.
Scraping page 3
Failed to retrieve job details: 429
37 scraped successfully.
Scraping page 4
47 scraped successfully.
Scraping page 5
57 scraped successfully.
Scraping page 6
Job filtered out
66 scraped successfully.
Scraping page 7
76 scraped successfully.
Scraping page 8
86 scraped successfully.
Scraping page 9
96 scraped successfully.
Scraping page 10
Job filtered out
105 scraped successfully.
Scraping page 11
115 scraped successfully.
Scraping page 12
Job filtered out
124 scraped successfully.
Scraping page 13
Job filtered out
132 scraped successfully.
Scraping page 14
Job filtered out
Job filtered out
140 scraped successfully.
Scraping page 15
Job filtered out
Job filtered out
Job filtered out
Job filtered out
146 scraped successfully.
Scraping page 16
156 scraped successfully.
Scraping page 17
Job filtered ou

In [ ]:
# Scrape job listings:
jobs = scrape.scrape_jobs(pages=2, job_title="Data Scientist", location="Chicago", post_time=5)

In [ ]:
# Save to JSON:
with open("data/job_queries/job_data_test.json", 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

In [4]:
# Load JSON:
with open("data/job_queries/Data-Scientist_2025-04-14_1.json", 'r', encoding='utf-8') as f:
    jobs = json.load(f)
raw_df = pd.DataFrame(jobs["jobs"])

In [ ]:
# Find job matches:
df = ana.find_job_match(models.embed_model, models.nlp_model, raw_df, "data/resume.pdf", models.stop_words, word_scores)

In [6]:
# generate job match report
html = ana.generate_html_report(df)